In [4]:
import numpy as np
import scipy
import scipy.stats
#import matplotlib as mpl
import matplotlib.pyplot as plt
#plt.style.use('classic')
#%matplotlib inline

In [5]:
from os import listdir
import os
from os.path import isfile, join

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# Estos son mis directorios
#root_dir = "/content/gdrive/My Drive/"
#base_dir = root_dir + 'Teo_info_proyecto_d'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
def teor_detcm_condit_other_vars_fn(covmat, n):
  m = []
  for i in range(0,n):
    covmat_less_one = np.delete(covmat, i, axis=0)
    covmat_less_one = np.delete(covmat_less_one, i, axis=1)
    teor_detcm_condit_other_vars_i = np.linalg.det(covmat_less_one)
    m.append(teor_detcm_condit_other_vars_i)
  return np.array(m)

In [7]:
# Ejemplo para alguna distribucion:
# Caso datos Gaussianos correlacionados
# Gaussiana multivariada N-dim

N = 3 # Dimensión
T = 10000 # Número de muestras a generar

# Generación matriz
covmat = np.random.rand(N,N)
covmat = 0.5*(covmat + np.transpose(covmat))
np.fill_diagonal(covmat,N)

# Media
mu = np.zeros(N)
covmat


array([[3.        , 0.58008716, 0.38829296],
       [0.58008716, 3.        , 0.26341498],
       [0.38829296, 0.26341498, 3.        ]])

In [9]:
DATA = np.random.multivariate_normal(mu,covmat,T)
DATA.shape

(10000, 3)

In [10]:
N = DATA.shape[1]
T = DATA.shape[0]
sort_id = DATA.argsort(axis=0) # Índices que ordenan los datos
sort_id = sort_id + 1

copdata = sort_id.argsort(axis=0) # Índices que ordenan los índices que ordenan los datos
copdata = copdata + 1
copdata = copdata / (T+1) # Normalización a [0,1]

# Usar inversa de CDF de la distribución normal estándar 
gaussian_data = scipy.stats.norm.ppf(copdata) # Datos uniformes a datos gaussianos
gaussian_data[np.isinf(gaussian_data)] = 0 # Remover infinitos

# Cálculo matriz de covarianza 
covmat_gc = np.matmul(np.transpose(gaussian_data),gaussian_data)/(T-1) 

# Medidas usando matriz obtenida por copulas

teor_detcm = np.linalg.det(covmat_gc) # Determinante matriz de covarianza
teor_detcm_condit_other_vars = teor_detcm_condit_other_vars_fn(covmat_gc, N) # Determinantes de matrices de covarianza sin considerar 1 variable
teor_single_vars = np.diagonal(covmat_gc) # Varianza de cada variable

#Entropias

teor_var_ents = 0.5*np.log(((2*np.pi*np.exp(1)))*teor_single_vars)# Entropía teórica de cada variable
teor_entropy = 0.5*(np.log((2*np.pi*np.exp(1))**N) + np.log(teor_detcm)) # Entropía conjunta teórica de la Gaussiana multivariante
teor_ent_condit_other_vars = 0.5*(np.log((2*np.pi*np.exp(1))**(N-1)) + np.log(teor_detcm_condit_other_vars)) # Entropía de 1 variable condicional a las otras otras variables

# TC, DTC, O-info, S-info (en nats)
teor_tc = sum(teor_var_ents) - teor_entropy
teor_dtc = sum(teor_ent_condit_other_vars) - (N-1)*teor_entropy
teor_oinfo = teor_tc - teor_dtc
teor_sinfo = teor_tc + teor_dtc

# TC, DTC, O-info (en bits)
teor_tc = round(teor_tc/np.log(2),4)
teor_dtc = round(teor_dtc/np.log(2),4)
teor_oinfo = round(teor_oinfo/np.log(2),4)

In [11]:
print('Las medidas de información calculadas por Gaussian copula (en bits) :')
print('Total correlation:   ' + str(teor_tc))
print('Dual total correlation:   ' + str(teor_dtc))
print('O-information:   ' + str(teor_oinfo))

Las medidas de información calculadas por Gaussian copula (en bits) :
Total correlation:   0.0412
Dual total correlation:   0.0387
O-information:   0.0025
